<a href="https://colab.research.google.com/github/varnikaaavyas/Time-Series-Analysis/blob/main/Vector_Autoregression_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
# turn off the warning
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import statsmodels.api as sm
data = sm.datasets.macrodata.load_pandas().data
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [ ]:
# stationarity tets
for (columnName, columnData) in data.iteritems():
  ad_fuller_tetst=adfuller(columnData)
  print(f'{columnName} P-value: {ad_fuller_tetst[1]}' )

# Real gdp and cpi P-value is greater than 0.05 . This is NOn-stationary


year P-value: 0.9325391339631047
quarter P-value: 0.0
realgdp P-value: 0.9982455372335032
realcons P-value: 0.9976992503412904
realinv P-value: 0.6484956579101143
realgovt P-value: 0.9558411923961085
realdpi P-value: 1.0
cpi P-value: 0.9904328188337422
m1 P-value: 0.9987380758531885
tbilrate P-value: 0.26991776442261584
unemp P-value: 0.10685366457233608
pop P-value: 0.9982010738893761
infl P-value: 0.030107620863486007
realint P-value: 0.007161872494651718


In [ ]:
fig = px.line(data, x=data.year, y = data.realgdp)
fig.show()

In [ ]:
fig = px.line(data, x=data.year, y = data.cpi)
fig.show()

In [ ]:
# stationarity tets
col = data[['realgdp','cpi']]
for (columnName, columnData) in col.iteritems():
  ad_fuller_tetst=adfuller(columnData)
  print(f'{columnName} P-value: {ad_fuller_tetst[1]}' )


realgdp P-value: 0.9982455372335032
cpi P-value: 0.9904328188337422


In [ ]:
data1 = col.diff()
data1.index=data['year']


In [ ]:
fig = px.line(data1, x=data.year, y = data1.columns)
fig.show()

In [ ]:
# stationarity tets
data1.dropna(inplace=True)
for (columnName, columnData) in data1.iteritems():
  ad_fuller_tetst=adfuller(columnData)
  print(f'{columnName} P-value: {ad_fuller_tetst[1]}' )

realgdp P-value: 3.327882187668224e-08
cpi P-value: 0.05891880557886097


In [ ]:

train_df=data1.iloc[:-10,:]
test_df=data1.iloc[-10:,:]
train_df

,realgdp,cpi
year,,
1959.0,68.452,0.170
1959.0,-3.313,0.200
1959.0,9.716,0.020
1960.0,62.495,0.170
1960.0,-13.309,0.010
...,...,...
2006.0,167.239,1.300
2006.0,46.524,2.000
2006.0,3.454,-0.800


In [ ]:
## training the model
model=VAR(train_df[1:])
sorted_order=model.select_order(maxlags=20)
print(sorted_order.summary())


 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        7.328       7.364       1522.       7.343
1        7.065       7.175       1170.       7.109
2        6.904       7.087       996.5       6.978
3       6.754*      7.010*      857.7*      6.858*
4        6.784       7.113       883.6       6.917
5        6.818       7.221       914.8       6.982
6        6.844       7.320       938.9       7.037
7        6.877       7.426       970.3       7.099
8        6.877       7.500       971.3       7.130
9        6.874       7.569       968.5       7.156
10       6.894       7.662       988.5       7.206
11       6.920       7.761       1015.       7.261
12       6.880       7.795       977.0       7.252
13       6.917       7.905       1014.       7.317
14       6.909       7.970       1008.       7.340
15       6.848       7.982       949.4       7.308
16       6.870       8.078     

In [ ]:
## fit model
var_model=VARMAX(train_df, order=(3, 0), enforce_stationarity=True)
fitted_model=var_model.fit()
print(fitted_model.summary())


                           Statespace Model Results                           
Dep. Variable:     ['realgdp', 'cpi']   No. Observations:                  192
Model:                         VAR(3)   Log Likelihood               -1164.346
                          + intercept   AIC                           2362.692
Date:                Tue, 11 Apr 2023   BIC                           2418.070
Time:                        04:40:59   HQIC                          2385.121
Sample:                             0                                         
                                - 192                                         
Covariance Type:                  opg                                         
Ljung-Box (L1) (Q):             0.00, 0.06   Jarque-Bera (JB):        14.58, 637.53
Prob(Q):                        0.95, 0.81   Prob(JB):                   0.00, 0.00
Heteroskedasticity (H):         1.63, 7.97   Skew:                       0.33, 0.82
Prob(H) (two-sided):            0.05,

In [ ]:
## forecasting
n_forecast=9
predict=fitted_model.get_prediction(start=len(train_df), end=len(train_df)+n_forecast)

In [ ]:
predictions=predict.predicted_mean
predictions.columns=['realgdp_pred','cpi_pred']
print(predictions)
test_df

     realgdp_pred  cpi_pred
192     33.612197  0.416358
193     51.100848  1.585249
194     47.977846  1.476511
195     40.909790  0.894945
196     51.093651  1.339312
197     47.398695  1.194611
198     46.944448  1.023840
199     50.372267  1.177057
200     48.995847  1.084463
201     49.436175  1.036341


,realgdp,cpi
year,,
2007.0,104.076,1.418
2007.0,117.132,1.795
2007.0,70.140,3.362
2008.0,-24.384,1.502
2008.0,48.401,4.613
2008.0,-90.666,-1.721
2008.0,-182.680,-4.715
2009.0,-216.510,0.497
2009.0,-23.906,1.798


In [ ]:
from sklearn.metrics import mean_squared_error
import math
cpi_pred =math.sqrt(mean_squared_error(predictions['cpi_pred'], test_df['cpi']))
print(f'CPI_RMSE={cpi_pred}')
realgdp_pred=math.sqrt(mean_squared_error(predictions['realgdp_pred'], test_df['realgdp']))
print(f'REALGDP_RMSE={realgdp_pred}')

CPI_RMSE=2.426280742034231
REALGDP_RMSE=128.0505551914587


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


In [ ]:
ts1 = np.array([3,4,5,5,7,9,13,15,12,10,8,8])
ts2 = np.array([21,19,22,24,25,29,30,34,37,40,35,30])


In [ ]:
ccv = np.cov(ts1,ts2)
print(ccv)

[[14.02272727 19.5       ]
 [19.5        45.60606061]]
